In [1]:
# Cell 1: Imports and Setup
import os
import pandas as pd

# Show all columns (instead of truncating)
pd.set_option('display.max_columns', None)

# Max width (in characters) of each column before truncating
#pd.set_option('display.max_colwidth', None)

# Floating-point precision
pd.set_option('display.precision', 4)

from datetime import date, timedelta
from dotenv import load_dotenv # Optional: for loading credentials from .env file

# Assuming your utils folder is in the same directory or you adjust the path
import sys
sys.path.append(os.path.abspath(os.path.join('.', 'utils'))) # Add utils to Python path

import garmin_utils
import data_processing

# Optional: For loading credentials safely if you use a .env file
# Create a .env file in your project root (and add it to .gitignore!)
# GARMIN_EMAIL="your_email@example.com"
# GARMIN_PASSWORD="your_password"
load_dotenv()
GARMIN_EMAIL = os.getenv("GARMIN_EMAIL")
GARMIN_PASSWORD = os.getenv("GARMIN_PASSWORD")

# --- Configuration for testing ---
TEST_USERNAME = GARMIN_EMAIL # Or hardcode for testing if not using .env
# Define a specific, short date range where you KNOW you have data on Garmin Connect
# For example, a single day or a few recent days.
TEST_START_DATE = date(2025, 3, 20) # Example: Year, Month, Day
TEST_END_DATE = date(2025, 5, 24)   # Example: Year, Month, Day
FORCE_REFRESH_FROM_GARMIN = True # Set to True to bypass cache initially

In [2]:
# Cell 2: Test Garmin Login
# Note: login_to_garmin is cached with @st.cache_resource in your app.
# In a notebook, this caching might behave differently or not apply if Streamlit isn't running.
# For notebook testing, you might call it directly without relying on Streamlit's caching.

print(f"Attempting to log in as {TEST_USERNAME}...")
try:
    # If login_to_garmin is heavily reliant on Streamlit's caching for resources,
    # you might need a slightly modified version for non-Streamlit contexts or
    # just use the direct Garmin client instantiation for debugging here.
    # For now, let's assume it can be called:
    garmin_client = garmin_utils.login_to_garmin(TEST_USERNAME, GARMIN_PASSWORD)
    if garmin_client:
        print("Login successful!")
    else:
        print("Login failed. Check credentials and Garmin Connect status.")
        # Stop execution if login fails
        raise Exception("Garmin Login Failed")
except Exception as e:
    print(f"Error during login attempt: {e}")
    garmin_client = None # Ensure it's None if login fails

Attempting to log in as alvarogonzalezdesande@gmail.com...


2025-05-26 17:03:24.854 
  command:

    streamlit run c:\Users\ag\alvaro\git\Garmin\venv_garmin\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-26 17:03:28,380 - INFO - Successfully logged in as alvarogonzalezdesande@gmail.com


Login successful!


In [3]:
# Cell 3: Test Fetching HRV Data (Raw)
if garmin_client:
    print(f"\nFetching HRV data for {TEST_USERNAME} from {TEST_START_DATE} to {TEST_END_DATE}...")
    # Directly call the core fetching logic within get_hrv_data if you want to see the absolute raw API response
    # For now, let's use the existing function which includes caching logic
    hrv_raw_df_from_util = garmin_utils.get_hrv_data(
        garmin_client,
        TEST_USERNAME,
        TEST_START_DATE,
        TEST_END_DATE,
        force_refresh=FORCE_REFRESH_FROM_GARMIN
    )
    print("\n--- HRV Data (from garmin_utils.get_hrv_data) ---")
    if not hrv_raw_df_from_util.empty:
        print(f"Shape: {hrv_raw_df_from_util.shape}")
        print(f"Columns: {hrv_raw_df_from_util.columns.tolist()}")
        print("Head:")
        print(hrv_raw_df_from_util.head())
        print("\nInfo:")
        hrv_raw_df_from_util.info()
    else:
        print("No HRV data returned by garmin_utils.get_hrv_data.")

    # You can also inspect the Parquet file if it was created
    hrv_cache_path = garmin_utils.get_user_data_path(
        TEST_USERNAME, "hrv",
        TEST_START_DATE.strftime("%Y-%m-%d"),
        TEST_END_DATE.strftime("%Y-%m-%d")
    )
    print(f"\nHRV Cache path: {hrv_cache_path}")
    if os.path.exists(hrv_cache_path):
        print("HRV Parquet file exists. Reading it directly:")
        try:
            df_hrv_parquet = pd.read_parquet(hrv_cache_path)
            print(f"Shape from Parquet: {df_hrv_parquet.shape}")
            print(df_hrv_parquet.head())
        except Exception as e:
            print(f"Error reading HRV Parquet: {e}")
    else:
        print("HRV Parquet file does NOT exist.")
else:
    print("Skipping HRV fetch due to login failure.")

2025-05-26 17:03:28,403 - INFO - Fetching HRV for alvarogonzalezdesande@gmail.com from Garmin API for range 2025-03-20 to 2025-05-24



Fetching HRV data for alvarogonzalezdesande@gmail.com from 2025-03-20 to 2025-05-24...


2025-05-26 17:03:38,582 - INFO - Raw HRV data fetched before DataFrame creation: []
2025-05-26 17:03:38,583 - INFO - No HRV data found for alvarogonzalezdesande@gmail.com for the period.



--- HRV Data (from garmin_utils.get_hrv_data) ---
No HRV data returned by garmin_utils.get_hrv_data.

HRV Cache path: data\alvarogonzalezdesande_gmail_com\hrv_2025-03-20_to_2025-05-24.parquet
HRV Parquet file exists. Reading it directly:
Shape from Parquet: (0, 0)
Empty DataFrame
Columns: []
Index: []


In [4]:
# Cell 4: Test Processing HRV Data
if garmin_client and not hrv_raw_df_from_util.empty:
    print("\n--- Processing HRV Data ---")
    hrv_processed_df = data_processing.process_hrv_df(hrv_raw_df_from_util.copy()) # Pass a copy
    if not hrv_processed_df.empty:
        print(f"Processed HRV Shape: {hrv_processed_df.shape}")
        print(f"Processed HRV Columns: {hrv_processed_df.columns.tolist()}")
        print("Processed HRV Head:")
        print(hrv_processed_df.head())
    else:
        print("HRV processing resulted in an empty DataFrame.")
        print("Check the logic in data_processing.process_hrv_df against the raw data structure printed above.")
elif garmin_client:
    print("Skipping HRV processing as raw HRV data was empty.")

Skipping HRV processing as raw HRV data was empty.


In [5]:
# Cell 5: Test Fetching Sleep Data (Raw)
if garmin_client:
    print(f"\nFetching Sleep data for {TEST_USERNAME} from {TEST_START_DATE} to {TEST_END_DATE}...")
    sleep_raw_df_from_util = garmin_utils.get_sleep_data(
        garmin_client,
        TEST_USERNAME,
        TEST_START_DATE,
        TEST_END_DATE,
        force_refresh=FORCE_REFRESH_FROM_GARMIN
    )
    print("\n--- Sleep Data (from garmin_utils.get_sleep_data) ---")
    if not sleep_raw_df_from_util.empty:
        print(f"Shape: {sleep_raw_df_from_util.shape}")
        print(f"Columns: {sleep_raw_df_from_util.columns.tolist()}")
        print("Head:")
        print(sleep_raw_df_from_util.head())
        print("\nInfo:")
        sleep_raw_df_from_util.info()
    else:
        print("No Sleep data returned by garmin_utils.get_sleep_data.")

    sleep_cache_path = garmin_utils.get_user_data_path(
        TEST_USERNAME, "sleep",
        TEST_START_DATE.strftime("%Y-%m-%d"),
        TEST_END_DATE.strftime("%Y-%m-%d")
    )
    print(f"\nSleep Cache path: {sleep_cache_path}")
    if os.path.exists(sleep_cache_path):
        print("Sleep Parquet file exists. Reading it directly:")
        try:
            df_sleep_parquet = pd.read_parquet(sleep_cache_path)
            print(f"Shape from Parquet: {df_sleep_parquet.shape}")
            print(df_sleep_parquet.head())
        except Exception as e:
            print(f"Error reading Sleep Parquet: {e}")
    else:
        print("Sleep Parquet file does NOT exist.")
else:
    print("Skipping Sleep fetch due to login failure.")

2025-05-26 17:03:38,672 - INFO - Fetching sleep data for alvarogonzalezdesande@gmail.com from Garmin API for range 2025-03-20 to 2025-05-24
2025-05-26 17:03:38,673 - ERROR - Error fetching sleep data range for alvarogonzalezdesande@gmail.com: 'Garmin' object has no attribute 'get_daily_sleep_data'
2025-05-26 17:03:38,674 - INFO - No sleep data found for alvarogonzalezdesande@gmail.com for the period.



Fetching Sleep data for alvarogonzalezdesande@gmail.com from 2025-03-20 to 2025-05-24...

--- Sleep Data (from garmin_utils.get_sleep_data) ---
No Sleep data returned by garmin_utils.get_sleep_data.

Sleep Cache path: data\alvarogonzalezdesande_gmail_com\sleep_2025-03-20_to_2025-05-24.parquet
Sleep Parquet file exists. Reading it directly:
Shape from Parquet: (0, 0)
Empty DataFrame
Columns: []
Index: []


In [6]:
# Cell 6: Test Processing Sleep Data
if garmin_client and not sleep_raw_df_from_util.empty:
    print("\n--- Processing Sleep Data ---")
    sleep_processed_df = data_processing.process_sleep_df(sleep_raw_df_from_util.copy()) # Pass a copy
    if not sleep_processed_df.empty:
        print(f"Processed Sleep Shape: {sleep_processed_df.shape}")
        print(f"Processed Sleep Columns: {sleep_processed_df.columns.tolist()}")
        print("Processed Sleep Head:")
        print(sleep_processed_df.head())
    else:
        print("Sleep processing resulted in an empty DataFrame.")
        print("Check the logic in data_processing.process_sleep_df against the raw data structure printed above.")
elif garmin_client:
    print("Skipping Sleep processing as raw sleep data was empty.")

Skipping Sleep processing as raw sleep data was empty.


In [7]:
# In your Jupyter Notebook, after successful login:
# garmin_client = garmin_utils.login_to_garmin(TEST_USERNAME, GARMIN_PASSWORD)

if garmin_client:
    print("--- Available methods on garmin_client (a selection) ---")
    # List all attributes and methods
    # for item in dir(garmin_client):
    #     if not item.startswith('_') and callable(getattr(garmin_client, item)): # List only callable methods
    #         print(item)

    # More targeted:
    print("\n--- Trying get_training_status (example for one day) ---")
    try:
        # Use a recent date where you likely had activity/status
        a_recent_date = (date.today() - timedelta(days=2)).isoformat()
        training_status = garmin_client.get_training_status(a_recent_date)
        print(f"Training Status for {a_recent_date}:")
        import json
        print(json.dumps(training_status, indent=2)) # Pretty print JSON
    except Exception as e:
        print(f"Error fetching training status: {e}")

    print("\n--- Trying get_spo2_data (example for one day) ---")
    try:
        spo2_data = garmin_client.get_spo2_data(a_recent_date)
        print(f"SpO2 Data for {a_recent_date}:")
        print(json.dumps(spo2_data, indent=2))
    except Exception as e:
        print(f"Error fetching SpO2 data: {e}")

    print("\n--- Trying get_respiration_data (example for one day) ---")
    try:
        resp_data = garmin_client.get_respiration_data(a_recent_date)
        print(f"Respiration Data for {a_recent_date}:")
        print(json.dumps(resp_data, indent=2))
    except Exception as e:
        print(f"Error fetching respiration data: {e}")

    # You can also try the problematic ones again for a single, known good date
    print("\n--- Re-trying get_hrv_data for a SINGLE specific date ---")
    # MAKE SURE THIS DATE HAS HRV DATA ON GARMIN CONNECT WEBSITE
    single_hrv_test_date = date(2024, 5, 23).isoformat() # <<< CHANGE TO A KNOWN GOOD DATE FOR YOUR HRV
    try:
        hrv_single = garmin_client.get_hrv_data(single_hrv_test_date)
        print(f"HRV Data for {single_hrv_test_date}:")
        print(json.dumps(hrv_single, indent=2))
    except Exception as e:
        print(f"Error fetching single date HRV: {e}")


    print("\n--- Re-trying get_sleep_data for a SINGLE specific date ---")
    # MAKE SURE THIS DATE HAS SLEEP DATA ON GARMIN CONNECT WEBSITE
    single_sleep_test_date = date(2024, 5, 23).isoformat() # <<< CHANGE TO A KNOWN GOOD DATE FOR YOUR SLEEP
    try:
        sleep_single = garmin_client.get_sleep_data(single_sleep_test_date)
        print(f"Sleep Data for {single_sleep_test_date}:")
        print(json.dumps(sleep_single, indent=2))
    except Exception as e:
        print(f"Error fetching single date sleep: {e}")

    print("\n--- Re-trying get_body_battery for a list containing a SINGLE specific date ---")
    # MAKE SURE THIS DATE HAS BODY BATTERY DATA ON GARMIN CONNECT WEBSITE
    single_bb_test_date_list = [date(2024, 5, 23).isoformat()] # <<< CHANGE TO A KNOWN GOOD DATE
    try:
        bb_single_list = garmin_client.get_body_battery(single_bb_test_date_list)
        print(f"Body Battery Data for {single_bb_test_date_list[0]}:")
        print(json.dumps(bb_single_list, indent=2))
    except Exception as e:
        print(f"Error fetching single date body battery: {e}")

else:
    print("Cannot explore client methods, login failed.")

--- Available methods on garmin_client (a selection) ---

--- Trying get_training_status (example for one day) ---
Training Status for 2025-05-24:
{
  "userId": 128204296,
  "mostRecentVO2Max": {
    "userId": 128204296,
    "generic": {
      "calendarDate": "2025-05-21",
      "vo2MaxPreciseValue": 53.9,
      "vo2MaxValue": 54.0,
      "fitnessAge": null,
      "fitnessAgeDescription": null,
      "maxMetCategory": 0
    },
    "cycling": null,
    "heatAltitudeAcclimation": {
      "calendarDate": "2025-05-21",
      "altitudeAcclimationDate": "2025-05-21",
      "previousAltitudeAcclimationDate": "2025-05-21",
      "heatAcclimationDate": "2025-05-21",
      "previousHeatAcclimationDate": "2025-05-21",
      "altitudeAcclimation": 0,
      "previousAltitudeAcclimation": 0,
      "heatAcclimationPercentage": 0,
      "previousHeatAcclimationPercentage": 0,
      "heatTrend": null,
      "altitudeTrend": null,
      "currentAltitude": 0,
      "previousAltitude": 0,
      "acclimati

In [8]:
garmin_client.get_stats(TEST_START_DATE)

{'userProfileId': 128204296,
 'totalKilocalories': 2346.0,
 'activeKilocalories': 593.0,
 'bmrKilocalories': 1753.0,
 'wellnessKilocalories': 2346.0,
 'burnedKilocalories': None,
 'consumedKilocalories': None,
 'remainingKilocalories': None,
 'totalSteps': 12557,
 'netCalorieGoal': None,
 'totalDistanceMeters': 17997,
 'wellnessDistanceMeters': 9153,
 'wellnessActiveKilocalories': 593.0,
 'netRemainingKilocalories': 593.0,
 'userDailySummaryId': 128204296,
 'calendarDate': '2025-03-20',
 'rule': {'typeId': 2, 'typeKey': 'private'},
 'uuid': '291ceba30b7b4f8db5d981870edb1435',
 'dailyStepGoal': 12620,
 'wellnessStartTimeGmt': '2025-03-19T23:00:00.0',
 'wellnessStartTimeLocal': '2025-03-20T00:00:00.0',
 'wellnessEndTimeGmt': '2025-03-20T23:00:00.0',
 'wellnessEndTimeLocal': '2025-03-21T00:00:00.0',
 'durationInMilliseconds': 86400000,
 'wellnessDescription': None,
 'highlyActiveSeconds': 540,
 'activeSeconds': 10373,
 'sedentarySeconds': 49747,
 'sleepingSeconds': 25740,
 'includesWellne

In [9]:
garmin_utils.get_daily_summaries(
        garmin_client,
        TEST_USERNAME,
        TEST_START_DATE,
        TEST_END_DATE,
        force_refresh=FORCE_REFRESH_FROM_GARMIN
    )

2025-05-26 17:03:40,722 - INFO - Fetching daily_summary for alvarogonzalezdesande@gmail.com from Garmin API for range 2025-03-20 to 2025-05-24
2025-05-26 17:03:51,601 - INFO - Saved daily_summary for alvarogonzalezdesande@gmail.com to cache: data\alvarogonzalezdesande_gmail_com\daily_summary_2025-03-20_to_2025-05-24.parquet


,userProfileId,totalKilocalories,activeKilocalories,bmrKilocalories,wellnessKilocalories,burnedKilocalories,consumedKilocalories,remainingKilocalories,totalSteps,netCalorieGoal,totalDistanceMeters,wellnessDistanceMeters,wellnessActiveKilocalories,netRemainingKilocalories,userDailySummaryId,calendarDate,rule,uuid,dailyStepGoal,wellnessStartTimeGmt,wellnessStartTimeLocal,wellnessEndTimeGmt,wellnessEndTimeLocal,durationInMilliseconds,wellnessDescription,highlyActiveSeconds,activeSeconds,sedentarySeconds,sleepingSeconds,includesWellnessData,includesActivityData,includesCalorieConsumedData,privacyProtected,moderateIntensityMinutes,vigorousIntensityMinutes,floorsAscendedInMeters,floorsDescendedInMeters,floorsAscended,floorsDescended,intensityMinutesGoal,userFloorsAscendedGoal,minHeartRate,maxHeartRate,restingHeartRate,lastSevenDaysAvgRestingHeartRate,source,averageStressLevel,maxStressLevel,stressDuration,restStressDuration,activityStressDuration,uncategorizedStressDuration,totalStressDuration,lowStressDuration,mediumStressDuration,highStressDuration,stressPercentage,restStressPercentage,activityStressPercentage,uncategorizedStressPercentage,lowStressPercentage,mediumStressPercentage,highStressPercentage,stressQualifier,measurableAwakeDuration,measurableAsleepDuration,lastSyncTimestampGMT,minAvgHeartRate,maxAvgHeartRate,bodyBatteryChargedValue,bodyBatteryDrainedValue,bodyBatteryHighestValue,bodyBatteryLowestValue,bodyBatteryMostRecentValue,bodyBatteryDuringSleep,bodyBatteryAtWakeTime,bodyBatteryVersion,abnormalHeartRateAlertsCount,averageSpo2,lowestSpo2,latestSpo2,latestSpo2ReadingTimeGmt,latestSpo2ReadingTimeLocal,averageMonitoringEnvironmentAltitude,restingCaloriesFromActivity,avgWakingRespirationValue,highestRespirationValue,lowestRespirationValue,latestRespirationValue,latestRespirationTimeGMT,respirationAlgorithmVersion,date
0,128204296,2346.0,593.0,1753.0,2346.0,None,None,None,12557,None,17997,9153,593.0,593.0,128204296,2025-03-20,"{'typeId': 2, 'typeKey': 'private'}",291ceba30b7b4f8db5d981870edb1435,12620,2025-03-19T23:00:00.0,2025-03-20T00:00:00.0,2025-03-20T23:00:00.0,2025-03-21T00:00:00.0,86400000,None,540,10373,49747,25740,True,True,False,False,63,26,46.293,46.510,15.1880,15.2592,400,10,37,144,47,47,GARMIN,18,99,7560,54000,11880,3060,76500,5880,1200,480,9.88,70.59,15.53,4.00,7.69,1.57,0.63,CALM,51000,22440,None,38,133,52,51,97,49,55,47,97,2.0,None,None,None,None,None,None,None,125.0,13.0,20.0,9.0,13.0,2025-03-20T23:00:00.0,100,2025-03-20
1,128204296,2280.0,527.0,1753.0,2280.0,None,None,None,11606,None,17116,8267,527.0,527.0,128204296,2025-03-21,"{'typeId': 2, 'typeKey': 'private'}",ba2d57d9dae14132ab60ff86ac226ec1,12620,2025-03-20T23:00:00.0,2025-03-21T00:00:00.0,2025-03-21T23:00:00.0,2025-03-22T00:00:00.0,86400000,None,1846,9332,51042,24180,True,True,False,False,64,22,60.089,59.046,19.7142,19.3721,400,10,42,139,46,47,GARMIN,17,81,7860,62640,12660,3000,86160,5220,2460,180,9.12,72.70,14.69,3.48,6.06,2.86,0.21,CALM,58980,24180,None,43,136,52,58,100,49,49,51,100,2.0,None,None,None,None,None,None,None,139.0,13.0,20.0,9.0,13.0,2025-03-21T23:00:00.0,100,2025-03-21
2,128204296,1995.0,242.0,1753.0,1995.0,None,None,None,7702,None,6020,6020,242.0,242.0,128204296,2025-03-22,"{'typeId': 2, 'typeKey': 'private'}",1b9eac085bdc4fcc9e47927e176b81a5,12420,2025-03-21T23:00:00.0,2025-03-22T00:00:00.0,2025-03-22T23:00:00.0,2025-03-23T00:00:00.0,86400000,None,2518,6327,52775,24780,True,False,False,False,1,5,23.577,25.943,7.7352,8.5115,400,10,45,124,47,47,GARMIN,27,96,26520,45720,11460,2640,86340,21240,4740,540,30.72,52.95,13.27,3.06,24.60,5.49,0.63,BALANCED,59220,24480,None,45,123,39,60,84,28,28,37,84,2.0,None,None,None,None,None,None,None,NaN,13.0,20.0,9.0,14.0,2025-03-22T22:58:00.0,100,2025-03-22
3,128204296,1971.0,218.0,1753.0,1971.0,None,None,None,5033,None,3919,3919,218.0,218.0,128204296,2025-03-23,"{'typeId': 2, 'typeKey': 'private'}",41209021f2334a52a71fc101cb817b24,11480,2025-03-22T23:00:00.0,2025-03-23T00:00:00.0,2025-03-23T23:00:00.

In [10]:
garmin_utils.get_activities(
        garmin_client,
        TEST_USERNAME,
        TEST_START_DATE,
        TEST_END_DATE,
        force_refresh=FORCE_REFRESH_FROM_GARMIN
    )

2025-05-26 17:03:51,653 - INFO - Fetching activities for alvarogonzalezdesande@gmail.com from Garmin API for range 2025-03-20 to 2025-05-24
2025-05-26 17:03:52,779 - INFO - Saved activities for alvarogonzalezdesande@gmail.com to cache: data\alvarogonzalezdesande_gmail_com\activities_2025-03-20_to_2025-05-24.parquet


,activityId,activityName,startTimeLocal,startTimeGMT,activityType,eventType,distance,duration,elapsedDuration,movingDuration,elevationGain,elevationLoss,averageSpeed,maxSpeed,startLatitude,startLongitude,hasPolyline,hasImages,ownerId,ownerDisplayName,ownerFullName,ownerProfileImageUrlSmall,ownerProfileImageUrlMedium,ownerProfileImageUrlLarge,calories,bmrCalories,averageHR,maxHR,averageRunningCadenceInStepsPerMinute,maxRunningCadenceInStepsPerMinute,steps,userRoles,privacy,userPro,hasVideo,timeZoneId,beginTimestamp,sportTypeId,avgPower,maxPower,aerobicTrainingEffect,anaerobicTrainingEffect,normPower,avgStrideLength,vO2MaxValue,workoutId,deviceId,minElevation,maxElevation,maxDoubleCadence,summarizedDiveInfo,maxVerticalSpeed,manufacturer,locationName,lapCount,endLatitude,endLongitude,waterEstimated,trainingEffectLabel,activityTrainingLoad,minActivityLapDuration,aerobicTrainingEffectMessage,anaerobicTrainingEffectMessage,splitSummaries,hasSplits,moderateIntensityMinutes,vigorousIntensityMinutes,avgGradeAdjustedSpeed,hasHeatMap,fastestSplit_1000,fastestSplit_1609,hrTimeInZone_1,hrTimeInZone_2,hrTimeInZone_3,hrTimeInZone_4,hrTimeInZone_5,powerTimeInZone_1,powerTimeInZone_2,powerTimeInZone_3,powerTimeInZone_4,powerTimeInZone_5,endTimeGMT,qualifyingDive,manualActivity,purposeful,pr,autoCalcCalories,elevationCorrected,atpActivity,favorite,decoDive,parent,fastestSplit_5000,fastestSplit_10000
0,19190751679,Madrid - Fartlek 4x,2025-05-21 12:22:17,2025-05-21 10:22:17,"{'typeId': 1, 'typeKey': 'running', 'parentTyp...","{'typeId': 9, 'typeKey': 'uncategorized', 'sor...",3213.8799,1137.5380,1138.6949,1132.701,59.0,72.0,2.825,4.376,40.4098,-3.6885,True,False,128204296,6d7ce41b-6f1f-4992-91c8-eaf232bfe0e6,Alvaro,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,215.0,26.0,142.0,157.0,159.7031,171.0,3018.0,"[SCOPE_GOLF_API_READ, SCOPE_ATP_READ, SCOPE_DI...","{'typeId': 2, 'typeKey': 'private'}",False,False,124,1747822937000,1,287.0,588.0,2.5,0.5,337.0,106.0293,54.0,1.2142e+09,3494773709,631.2,663.8,171.0,{'summarizedDiveGases': []},0.6000,GARMIN,Madrid,10,40.4085,-3.6915,300.0,TEMPO,48.4596,60.000,MAINTAINING_TEMPO_21,NO_ANAEROBIC_BENEFIT_0,"[{'noOfSplits': 1, 'totalAscent': 0.0, 'durati...",True,0.0,18.0,2.787,False,333.225,550.990,20.641,71.989,990.531,47.999,0.000,241.061,106.953,32.996,29.004,201.495,2025-05-21 10:41:15,False,False,False,False,False,False,False,False,False,False,NaN,NaN
1,19154483143,Madrid Walking,2025-05-17 18:45:52,2025-05-17 16:45:52,"{'typeId': 9, 'typeKey': 'walking', 'parentTyp...","{'typeId': 9, 'typeKey': 'uncategorized', 'sor...",1984.4200,1905.1121,1905.1121,1703.000,71.0,1.0,1.042,1.446,40.3985,-3.7032,True,False,128204296,6d7ce41b-6f1f-4992-91c8-eaf232bfe0e6,Alvaro,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,133.0,43.0,90.0,104.0,77.0469,159.0,2856.0,"[SCOPE_GOLF_API_READ, SCOPE_ATP_READ, SCOPE_DI...","{'typeId': 2, 'typeKey': 'private'}",False,False,124,1747500352000,11,NaN,NaN,0.3,0.0,NaN,81.4057,54.0,NaN,3494773709,593.2,663.4,159.0,{'summarizedDiveGases': []},0.2500,GARMIN,Madrid,2,40.4124,-3.7050,295.0,UNKNOWN,3.7054,913.844,NO_AEROBIC_BENEFIT_18,NO_ANAEROBIC_BENEFIT_0,[],False,28.0,0.0,NaN,False,NaN,NaN,323.840,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,2025-05-17 17:17:37,False,False,False,False,False,False,False,False,False,False,NaN,NaN
2,19141211993,Madrid Running,2025-05-16 12:09:55,2025-05-16 10:09:55,"{'typeId': 1, 'typeKey': 'running', 'parentTyp...","{'typeId': 9, 'typeKey': 'uncategorized', 'sor...",7081.8101,2485.4250,2498.9700,2478.906,26.0,26.0,2.849,4.152,40.4105,-3.6871,True,False,128204296,6d7ce41b-6f1f-4992-91c8-eaf232bfe0e6,Alvaro,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,538.0,

In [11]:
garmin_client.get_personal_records()

AttributeError: 'Garmin' object has no attribute 'get_personal_records'